![alt text](DataKind_orange.png)

### Multiclass Classifier for contract clause types

In [1]:
import os, io, requests
from collections import Counter
from itertools import product
import unicodedata
import re, string
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from textblob import TextBlob
from langdetect import detect

from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('notebook', rc={'figure.figsize': (10, 6)}, font_scale=1.5)

In [2]:
# spacy is used for Part of Speech tagging and Named Entity Recognition
# spacy is a non-standard python library which can be installed using 'pip install spacy' from the command line
# language models can be downloaded by running 'python -m spacy download <language>' from the command line
import spacy
supported_languages = ['en','fr','es']
nlp_langs = {}
for language in supported_languages:
    nlp_langs[language]  = spacy.load(language)

In [3]:
# Downloaded xls file annotations from resourcecontracts and openlandcontracts
resource_folder = 'contract_data/Contracts_Annotations/resource_contracts/'
land_folder = 'contract_data/Contracts_Annotations/openland_contracts/'
# Most recently downloaded metadata from resourcecontracts.org/contracts
rc_metadata = 'contract_data/resource_contract_2017-08-16.csv' 
# Most recently downloaded metadata from openlandcontracts.org/contracts
ol_metadata = 'contract_data/openland_contract_2017-08-16.csv'
metadata_files = [rc_metadata,ol_metadata]
folders = [resource_folder,land_folder]

In [4]:
labels = ['stabilization','royalties'] # list in priority order in case of duplicates
n_folds = 5

In [ ]:
# Languages must be supported by NLTK Snowball Stemmer and stopwords
languages = ['english','french','spanish','italian','portuguese','dutch','swedish','german']
s = requests.get('http://data.okfn.org/data/core/language-codes/r/language-codes.csv').content
language_codes = pd.read_csv(io.StringIO(s.decode('utf-8')))
language_codes.columns = ['language_code','DetectedLanguage']

In [ ]:
annotations = pd.DataFrame()
for folder in folders:
    xls_files = [f for f in os.listdir(folder) if f.lower().endswith('.xls')]
    for xls in xls_files:
        temp = pd.read_excel(folder + xls)
        if len(temp) > 0:
            temp['OCID'] = xls[:-4]
            temp['Source'] = folder.split('/')[-2]
            annotations = annotations.append(temp)
print "Number of annotations: " + str(len(annotations))

In [ ]:
# drop blank and integer annotations and annotations less than 4 words
annotations.dropna(subset=['Annotation Text'],inplace=True)
annotations = annotations[annotations['Annotation Text'].apply(lambda x: type(x)!=int)].copy()
annotations = annotations[annotations['Annotation Text'].apply(lambda x: len(x.split()) > 3)].copy()
print len(annotations)
# If duplicate text appears within the same contract, drop it
annotations.drop_duplicates(['Annotation Text','OCID','Category'],inplace=True)
print len(annotations)

In [ ]:
labels = [label.lower() for label in labels]
annotations['label'] = [x.lower() if x.lower() in labels else 'other' for x in annotations['Category']]

In [ ]:
# Given some duplicate text with different Category labels, keep the labels that are in our target label list when dropping
sort_num = range(len(labels) + 1)
sort_key = dict(zip(labels,sort_num))
sort_key['other'] = sort_num[-1]
annotations['sort_key'] = [sort_key[x] for x in annotations['label']]
annotations.sort_values(by='sort_key',inplace=True,ascending=True)
annotations.drop_duplicates(['Annotation Text','OCID'],keep='first',inplace=True)
print len(annotations)

In [ ]:
# Join metadata from contracts repository
metadata = pd.DataFrame()
for filename in metadata_files:
    temp = pd.read_csv(filename)
    metadata = metadata.append(temp)
    
annotations = annotations.merge(metadata[['OCID','Language','Country Name','Resource','Contract Type','Document Type']],how='left',on='OCID')

In [ ]:
y = list(annotations['label'])
yunique = list(np.unique(y))
for item in yunique:
    print str("{0:.2f}%".format(100*y.count(item) / float(len(y)))) + " " + item

In [ ]:
annotations.head(2)

In [ ]:
def punctuation_remove(text):
    """
    Mutates and returns text where all punctuation besides underscores,
    are replaced
    """
    punctuation_re = r'[^\w\s_]'
    new_text = nltk.regexp.re.sub(punctuation_re, ' ', text)
    return new_text

def replace_numbers(text):
    ''' 
    Removes all characters but periods, commas and alpha-numeric and 
    returns all numeric values replace with the word numeric_value
    '''
#     allowed = {",", "."," ","%"}.union(string.ascii_letters).union([str(num) for num in range(0,10)])
#     filtered = ''.join([character for character in text if character in allowed])
    wordlist = text.split()
    for i in range(len(wordlist)):
        if '$' in wordlist[i]:
            try:
                int(wordlist[i].split('$')[-1].replace(',','').replace('.','').replace('-','').replace(')','').replace('(',''))
                wordlist[i] = ' '.join(wordlist[i].split('$')[:-1]) + ' dollar_value'
            except:
                pass
        elif u'\xb0' in wordlist[i]:
            wordlist[i] == 'degree_value'
        elif '%' in wordlist[i]:
            try: 
                int(wordlist[i].replace(',','').replace('.','').replace('-','').replace('/','').replace('%','').replace(')','').replace('(',''))
                wordlist[i] = 'percent_value'
            except:
                pass
        else:
            try: 
                int(wordlist[i].replace(',','').replace('.','').replace('-','').replace('/','').replace(')','').replace('(',''))
                wordlist[i] = 'numeric_value'
            except:
                pass
    return ' '.join(wordlist)

def perform_lowercase(text):
    """
    Mutates and returns text where all characters are lowercased
    """
    try:
        new_text = text.lower()
    except:
        new_text = str(text).lower()
    return new_text

def underscore_remove(text):
    '''
    replaces multiple underscores with text fillintheblank
    and single underscore with space
    '''
    double_underscore_re = r'(__[a-zA-Z0-9_]*(__)?)'
    text = nltk.regexp.re.sub(double_underscore_re,'fillintheblank',text)
    return text.replace('_',' ')

def doublespace_remove(text):
    return re.sub(' +',' ',text)

def textblobsent(text):
    '''
    returns the TextBlob polarity and subjectivity
    '''
    text = text.encode('ascii','replace')
    sent = TextBlob(text).sentiment
    return pd.Series([sent.polarity,sent.subjectivity])

def get_avg_wordlength(document):
    wordlengths = [len(word) for word in document.split()]
    if len(wordlengths) == 0:
        return 0
    else:
        return np.mean(wordlengths)

def multilingual_pos(text,language):
    if language in nlp_langs.keys():
        try:
            tokens = nlp_langs[language](text)
            tags = [token.pos_ for token in tokens]
            counts = Counter(tags).items()
            countdict = {}
            for key, value in counts:
                countdict[key] = value
            return countdict
        except:
            print type(text), text
    else:
        return {'unsupported_language':1}
    
def detect_lang(text):
    '''
    Returns detected language
    '''
    text = doublespace_remove(text)
    try:
        return detect(text)
    except:
        return 'None'
    
def remove_stopwords(row):
    '''
    Multilingual stopwords removal
    '''
    language = row['Language']
    if language in languages:
        text = ' '.join([word for word in row['CleanText'].split(' ') if word not in stopwords.words(language)])
        return text
    else:
        return row['CleanText']
    
def stem_words(row):
    ''' 
    Multilingual word stemmer
    '''
    language = row['Language']    
    if language in languages:
        stemmer = SnowballStemmer(language)
        text = ' '.join([stemmer.stem(word) for word in row['CleanText_NoStop'].split(' ')])
        return text
    else:
        return row['CleanText_NoStop']
    
def clean_metadata(text):
    if type(text) in [float,int]:
        return text
    elif type(text) == str:
        return text.lower().split(';')[0]
    else:
        text = text.encode('ascii','replace')
        return text.lower().split(';')[0]

In [ ]:
def clean_text(df):
    df['Annotation Text'].fillna('',inplace=True)
    df['CleanText'] = df['Annotation Text']
    func_list = [perform_lowercase,replace_numbers,punctuation_remove,underscore_remove, doublespace_remove]
    for func in func_list:
        df['CleanText'] = df['CleanText'].apply(func)

    return df

def featurize(df):
    
    df['AvgWordLength'] = df['CleanText'].apply(get_avg_wordlength)
    df['language_code'] = df['CleanText'].apply(detect_lang)
    df = df.merge(language_codes,how='left',on='language_code')
    df['DetectedLanguage'] = df['DetectedLanguage'].astype('str')
    df['DetectedLanguage'] = df['DetectedLanguage'].apply(lambda x: x.lower().split(';')[0])
    df['DetectedLanguage'] = df['DetectedLanguage'].apply(lambda x: x if x in languages else 'other')
    df['DetectedLanguage'].fillna('None')
    langdummies = pd.get_dummies(df['DetectedLanguage'],prefix='detected_language')
    # Drop least frequent language
    langdummies.drop([col for col, val in langdummies.sum().iteritems() if val == langdummies.sum().min()],axis=1,inplace=True)
    
    df['CleanText_NoStop'] = df.apply(remove_stopwords,axis=1)
    df['CleanText_NoStop_Stemmed'] = df.apply(stem_words,axis=1)
    df.drop(['language_code','DetectedLanguage'],axis=1,inplace=True)
    
    postagcounts = []
    for index, row in df.iterrows():
        postagcounts.append(multilingual_pos(row['Annotation Text'],row['Language']))    
    postagdf = pd.DataFrame(postagcounts).fillna(0)
    postagdf.index = df.index
    postagdf.columns = ['postag_' + col for col in postagdf.columns]

    # create dummy variables for categoricals
    df['Resource'] = df['Resource'].apply(clean_metadata)
    df['Contract Type'] = df['Contract Type'].apply(clean_metadata)
    df['Document Type'] = df['Document Type'].apply(lambda x: x.lower().split(';')[0])
    dummy_cols = ['Language','Country Name','Resource','Contract Type','Document Type']
    dummies = pd.get_dummies(df[dummy_cols],prefix = dummy_cols)
    # drop lowest least frequent dummy columns for each
    for dummy_col in dummy_cols:
        cols = [col for col in dummies.columns if col.startswith(dummy_col)]
        dummies.drop([col for col, val in dummies[cols].sum().iteritems() if val == dummies[cols].sum().min()],axis=1,inplace=True)
    df.drop(dummy_cols, axis=1,inplace=True)
        
    textblobsentdf = df['CleanText'].apply(textblobsent)
    textblobsentdf.columns = ['TextblobPolarity','TextblobSubjectivity']
    df = pd.concat([df,textblobsentdf,langdummies,postagdf,dummies],axis=1)
    
    return df

In [ ]:
annotations = clean_text(annotations)
annotations = featurize(annotations)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df= .6,
                                 min_df= .001, 
                                 stop_words=None,  
                                 use_idf=True, 
                                 ngram_range=(1,4))
tfidf_matrix = tfidf_vectorizer.fit_transform(annotations['CleanText_NoStop_Stemmed'].values.astype('U'))
terms = tfidf_vectorizer.get_feature_names()
tfidf_matrix = tfidf_matrix.todense()
tfidf = pd.DataFrame(tfidf_matrix)
tfidf.index = annotations.index
tfidf.columns = terms
# tfidf.columns = [term.encode('ascii','replace') for term in tfidf.columns]
print tfidf.shape

In [ ]:
exclude = ['Source','Category','Topic','Annotation Text','CleanText','CleanText_NoStop','CleanText_NoStop_Stemmed',
           'OCID','PDF Page Number','Article Reference','MD','VBP','VBZ','VBG','VBD','VBN','other',"''",'label','sort_key']
features = [str(col) for col in annotations.columns.tolist() if not col in exclude]

In [ ]:
numeric_terms = []
for term in terms:
    for word in term.split():
        try:
            int(word)
            if not term in numeric_terms:
                numeric_terms.append(term)
        except:
            pass

In [ ]:
# 10                           183
# sec 11                       188
# 000                          232
# 11, 288

In [ ]:
tfidf[numeric_terms].apply(lambda column: (column != 0).sum()).sort_values()

In [ ]:
print len(numeric_terms)

In [ ]:
replace_numbers('MOPP will invest US$64,000,000,000. US$48,000,000,000 will be for the development of the concession area, and US$16,000,000,000 will be for the outgrowers program. The investment for the concession area will be divided as follows: initial investment of US$15,000,000,000 during the first numeric_value years (which includes investments prior to date the contract comes into force), second investment of US$ numeric_value over a second numeric_value year term, and a final US$ numeric_value during the rest of the term of the contract.')

In [ ]:
test = annotations[['OCID','Category','Annotation Text','CleanText']].iloc[8473]
print test[['OCID','Category']]
print test['CleanText']
print '\n'
print replace_numbers(test['Annotation Text'])

In [ ]:
tfidf[tfidf['000 000'] > 0]['000 000'].sort_values(ascending=False)

In [ ]:
X = pd.concat([tfidf,annotations[features]],axis=1)

In [ ]:
X.fillna(0,inplace=True)
X = X.rename(columns = {'fit':'fit_feature'})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
model = OneVsRestClassifier(RandomForestClassifier(
                                                    n_estimators=500,
                                                    max_features=.2, 
                                                    min_samples_leaf=50,
                                                    class_weight='balanced_subsample',
                                                    min_samples_split=2,
                                                    bootstrap=True,
                                                    max_depth=None, 
                                                    n_jobs=-1))

In [ ]:
print str(n_folds) + ' Fold CV Accuracy Score: ' + str(np.mean(cross_val_score(model, X, y, cv=n_folds)))
model.fit(X_train, y_train)
test_pred = model.predict(X_test)

In [ ]:
print "Test Set Classification Report \n" + classification_report(y_test, test_pred, target_names=yunique)
print "Test Set Accuracy: " + str(accuracy_score(y_test, test_pred))

In [ ]:
cnf_matrix = confusion_matrix(y_test, test_pred)
np.set_printoptions(precision=2)

# Plot confusion matrix
sns.set_context(rc={"figure.figsize": (8, 8)})       
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=model.classes_,
                          title='Confusion matrix ');

In [ ]:
def get_feature_importances(clf,X):
    '''
    returns feature importances
    '''
    classes = clf.classes_
    feat_importance = {}
    for i in range(len(classes)):
        importances = clf.estimators_[i].feature_importances_
        indices = np.argsort(importances)[::-1]
        feat_importance[classes[i]] = pd.DataFrame(X.columns, importances)    
        feat_importance[classes[i]].reset_index(inplace=True)
        feat_importance[classes[i]].columns = ['Importance','Feature']
        feat_importance[classes[i]].sort_values(by='Importance', axis=0, ascending=True, inplace=True)
        feat_importance[classes[i]].set_index('Feature',inplace=True)
    return feat_importance

In [ ]:
n = 20
sns.set_context(rc={"figure.figsize": (8, 6)})       
importances = {}
importances = get_feature_importances(model,X)
for i in model.classes_[1:]:
    print 'Top ' + str(n) + ' Features for ' + i
    importances[i].index.name=None 
    importances[i].sort_values(by='Importance',ascending=False).head(n).plot(kind='barh',title=i.title() + ' Top ' + str(n) + ' Features',legend=False, color='lightblue')
    plt.tight_layout()
    plt.show()

In [ ]:
annotations['Actual'] = y
annotations['Predicted'] = model.predict(X)

In [ ]:
missed = len(annotations[annotations['Predicted'] != annotations['Actual']])
hit = len(annotations) - missed
print 'Hit: ' + str(hit)
print 'Missed: ' + str(missed)
print 'Accuracy: ' + str(round(hit / float(len(annotations)),3))

In [ ]:
false_positives = annotations[(annotations['Predicted'] != annotations['Actual']) 
                              & (annotations['Predicted'] != 'other')]
false_negatives = annotations[(annotations['Predicted'] != annotations['Actual']) 
                              & (annotations['Predicted'] == 'other')]
true_positives = annotations[(annotations['Predicted'] == annotations['Actual']) 
                              & (annotations['Predicted'] != 'other')]
print 'False Positives: ' + str(len(false_positives))
print 'False Negatives: ' + str(len(false_negatives))

In [ ]:
false_positives[false_  positives['language_english'] == 1][['OCID','Category','Predicted','CleanText']].to_csv('classification_results/False_Positives.csv')

In [ ]:
false_negatives[['OCID','Category','Predicted','CleanText']].to_csv('classification_results/False_Negatives.csv')